## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [1]:
import pandas as pd
import numpy as np

#1
filename = '/home/alberto/data/data_000637.txt'
num_lines = sum(1 for line in open(filename))
N = np.random.randint(1e4, num_lines+1)
data = pd.read_csv(filename, nrows=N)
print(num_lines)
data

1310721


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
241422,1,1,8,3869202569,2091,12
241423,1,0,56,3869202569,2110,20
241424,1,1,20,3869202569,2091,12
241425,1,0,61,3869202569,2117,7


In [2]:
#2
data_group_orbit = data.groupby(['ORBIT_CNT'])['BX_COUNTER']
bx_cnts_max = data_group_orbit.max().max()+1
freq = sum(1 for item in data_group_orbit.max() if item == bx_cnts_max-1)
print("BX_COUNTS MAX:", bx_cnts_max)
print("Encountered", freq, "times")

BX_COUNTS MAX: 3564
Encountered 44 times


In [3]:
#3
data_orb_max = data[data['ORBIT_CNT']==data['ORBIT_CNT'].max()]
data_orb_bx_max = data_orb_max[data_orb_max['BX_COUNTER']==data_orb_max['BX_COUNTER'].max()]
time = ((data['ORBIT_CNT'].max() - data['ORBIT_CNT'].min())*bx_cnts_max + data_orb_max['BX_COUNTER'].max())*25 + data_orb_bx_max['TDC_MEAS'].max()*5/6
print("Data taken for", time, "ns")

Data taken for 214071130.83333334 ns


In [4]:
#4
data['TIME_ns'] = pd.Series( (data['ORBIT_CNT']*bx_cnts_max+data['BX_COUNTER'])*25 + data['TDC_MEAS']*5/6, index = data.index)
print("Last registered time:", data['TIME_ns'].values[-1])
data

Last registered time: 344745948950236.7


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_ns
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,1,0,63,3869200167,2553,28,3.447457e+14
3,1,0,64,3869200167,2558,19,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
241422,1,1,8,3869202569,2091,12,3.447459e+14
241423,1,0,56,3869202569,2110,20,3.447459e+14
241424,1,1,20,3869202569,2091,12,3.447459e+14
241425,1,0,61,3869202569,2117,7,3.447459e+14


In [13]:
#5
data['HEAD'] = pd.Series(np.random.randint(2, size=data['HEAD'].size), index=data.index)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_ns
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,1,0,63,3869200167,2553,28,3.447457e+14
3,0,0,64,3869200167,2558,19,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
241422,0,1,8,3869202569,2091,12,3.447459e+14
241423,1,0,56,3869202569,2110,20,3.447459e+14
241424,0,1,20,3869202569,2091,12,3.447459e+14
241425,0,0,61,3869202569,2117,7,3.447459e+14


In [16]:
#6
new_data = data[data['HEAD']==1].copy()
new_data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_ns
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,1,0,63,3869200167,2553,28,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
9,1,0,60,3869200167,2788,7,3.447457e+14
...,...,...,...,...,...,...,...
241415,1,0,139,3869202569,2091,0,3.447459e+14
241417,1,1,6,3869202569,2090,21,3.447459e+14
241418,1,1,7,3869202569,2086,17,3.447459e+14
241421,1,1,21,3869202569,2086,25,3.447459e+14


In [ ]:
#7
